In [1]:
from delta import configure_spark_with_delta_pip, DeltaTable
from pyspark.sql import SparkSession

In [2]:
builder = (SparkSession.builder
           .appName("change-data-feed-delta-table")
           .master("spark://spark-master:7077")
           .config("spark.executor.memory", "512m")
           .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
           .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog"))

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-664c02ec-fd6c-4a3a-932a-b171dfceab60;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 280ms :: artifacts dl 11ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0 

In [3]:
%load_ext sparksql_magic
%config SparkSql.limit=20

### Create Bronze Table 
(as an appendOnly table)

In [4]:
%%sparksql
CREATE OR REPLACE TABLE default.movie_and_show_titles_cdf (
    show_id STRING,
    type STRING,
    title STRING,
    director STRING,
    cast STRING,
    country STRING,
    date_added STRING,
    release_year STRING,
    rating STRING,
    duration STRING,
    listed_in STRING,
    description STRING 
) USING DELTA LOCATION '/opt/workspace/data/delta_lake/movie_and_show_titles_cdf'
TBLPROPERTIES (delta.enableChangeDataFeed = true, medallionLevel = 'bronze');

#### Initial Load of Bronze Table

In [5]:
# Read CSV file into a DataFrame
df = (spark.read
      .format("csv")
      .option("header", "true")
      .load("../data/netflix_titles.csv"));
df.write.format("delta").mode("append").saveAsTable("default.movie_and_show_titles_cdf")

In [6]:
%%sparksql
CREATE OR REPLACE TABLE default.movie_and_show_titles_cleansed (
    show_id STRING,
    type STRING,
    title STRING,
    director STRING,
    cast STRING,
    country STRING,
    date_added STRING,
    release_year STRING,
    rating STRING,
    duration STRING,
    listed_in STRING,
    description STRING 
) USING DELTA LOCATION '/opt/workspace/data/delta_lake/movie_and_show_titles_cleansed'
TBLPROPERTIES (delta.enableChangeDataFeed = true, medallionLevel = 'silver'
, updatedFromTable= 'default.movie_and_show_titles_cdf', updatedFromTableVersion= '-1');

In [7]:
#get the value of the last Updated Version from the silver table
lastUpdateVersion = int(spark.sql("SHOW TBLPROPERTIES default.movie_and_show_titles_cleansed ('updatedFromTableVersion')").first()["value"])+1
lastUpdateVersion

0

In [8]:
#get the value of the last Updated Version from the silver table
latestVersion = spark.sql("DESCRIBE HISTORY default.movie_and_show_titles_cdf").first()["version"]
latestVersion

1

#### Create temp view of chnage to bronze table since last load of silver

In [9]:
%%sparksql
CREATE OR REPLACE TEMPORARY VIEW bronzeTable_latest_version as
SELECT * FROM (
    SELECT *, 
        RANK() OVER (
        PARTITION BY (lower(type), lower(title), lower(director), date_added) 
        ORDER BY _commit_version DESC) as rank  
    FROM table_changes('default.movie_and_show_titles_cdf',{lastUpdateVersion},{latestVersion})
    WHERE type IS NOT NULL AND title IS NOT NULL AND director IS NOT NULL AND  _change_type !='update_preimage'
)
WHERE rank = 1;

### Merge Change Data into Silver

In [10]:
%%sparksql 
MERGE INTO default.movie_and_show_titles_cleansed t 
USING bronzeTable_latest_version s 
ON lower(t.type) = lower(s.type)
AND lower(t.title) = lower(s.title)
AND lower(t.director) = lower(s.director)
AND t.date_added = s.date_added
WHEN MATCHED AND s._change_type='update_postimage' OR s._change_type='update_postimage' THEN UPDATE SET *
WHEN MATCHED AND s._change_type='delete' THEN DELETE
WHEN NOT MATCHED AND s._change_type='insert' THEN INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
6172,0,0,6172


In [11]:
%%sparksql
ALTER TABLE default.movie_and_show_titles_cleansed SET TBLPROPERTIES(updatedFromTableVersion = {latestVersion});

In [12]:
%%sparksql
DROP VIEW bronzeTable_latest_version

### Update Bronze Table

In [13]:
%%sparksql
DELETE FROM default.movie_and_show_titles_cdf WHERE country is NULL

num_affected_rows
830


In [14]:
%%sparksql
UPDATE default.movie_and_show_titles_cdf SET director = '' WHERE director is NULL

num_affected_rows
2226


### Propogate Changes from Bronze to Silver

In [15]:
#get the value of the last Updated Version from the silver table
lastUpdateVersion = spark.sql("SHOW TBLPROPERTIES default.movie_and_show_titles_cleansed ('updatedFromTableVersion')").first()["value"]
lastUpdateVersion

'1'

In [16]:
#get the value of the last Updated Version from the silver table
latestVersion = spark.sql("DESCRIBE HISTORY default.movie_and_show_titles_cdf").first()["version"]
latestVersion

3

In [17]:
%%sparksql
CREATE OR REPLACE TEMPORARY VIEW bronzeTable_latest_version as
SELECT * FROM (
    SELECT *, 
        RANK() OVER (
        PARTITION BY (lower(type), lower(title), lower(director), date_added) 
        ORDER BY _commit_version DESC) as rank  
    FROM table_changes('default.movie_and_show_titles_cdf',{lastUpdateVersion},{latestVersion})
    WHERE type IS NOT NULL AND title IS NOT NULL AND director IS NOT NULL AND  _change_type !='update_preimage'
)
WHERE rank = 1;

In [18]:
%%sparksql
SELECT _change_type, COUNT(*) FROM bronzeTable_latest_version GROUP BY _change_type

_change_type,count(1)
update_postimage,2226
delete,422
insert,5750


In [19]:
%%sparksql 
MERGE INTO default.movie_and_show_titles_cleansed t 
USING bronzeTable_latest_version s 
ON lower(t.type) = lower(s.type)
AND lower(t.title) = lower(s.title)
AND lower(t.director) = lower(s.director)
AND t.date_added = s.date_added
WHEN MATCHED AND s._change_type='update_postimage' THEN UPDATE SET *
WHEN MATCHED AND s._change_type='delete' THEN DELETE
WHEN NOT MATCHED AND s._change_type='insert' THEN INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
422,0,422,0


In [20]:
%%sparksql
ALTER TABLE default.movie_and_show_titles_cleansed SET TBLPROPERTIES(updatedFromTableVersion = {latestVersion});

In [21]:
spark.stop()